# Overview

This is a scratch notebook, where I conducted much of my data exploration - finding column data types, creating sub-sets of the data, combining the Texas Department of State Health Services overdose death data, etc.

## Washington Post Data

Source: [Washington Post DEA Database](https://www.washingtonpost.com/graphics/2019/investigations/dea-pain-pill-database/)

In [ ]:
# Imports

# Data import and manipulation
import pandas as pd
# Math
import numpy as np
# Let's go ahead and seed the notebook, for reproducibility
np.random.seed(113)

In [ ]:
# After a brief exploration, these are the datatypes per column of the
# Washington Post dataset. Making them explicit for speed here
# Note this still takes for-ever to load
dtypes = {
    "REPORTER_DEA_NO" : "object",
    "REPORTER_BUS_ACT" : "object",
    "REPORTER_NAME" : "object",
    "REPORTER_ADDL_CO_INFO" : "object",
    "REPORTER_ADDRESS1" : "object",
    "REPORTER_ADDRESS2" : "object",
    "REPORTER_CITY" : "object",
    "REPORTER_STATE" : "object",
    "REPORTER_ZIP" : "int64",
    "REPORTER_COUNTY" : "object",
    "BUYER_DEA_NO" : "object",
    "BUYER_BUS_ACT" : "object",
    "BUYER_NAME" : "object",
    "BUYER_ADDL_CO_INFO" : "object",
    "BUYER_ADDRESS1" : "object",
    "BUYER_ADDRESS2" : "object",
    "BUYER_CITY" : "object",
    "BUYER_STATE" : "object",
    "BUYER_ZIP" : "int64",
    "BUYER_COUNTY" : "object",
    "TRANSACTION_CODE" : "object",
    "DRUG_CODE" : "int64",
    "NDC_NO" : "object",
    "DRUG_NAME" : "object",
    "QUANTITY" : "float64",
    "UNIT" : "float64",
    "ACTION_INDICATOR" : "object",
    "ORDER_FORM_NO" : "object",
    "CORRECTION_NO" :  "float64",
    "STRENGTH" : "float64",
    "TRANSACTION_DATE" : "int64",
    "CALC_BASE_WT_IN_GM" : "float64",
    "DOSAGE_UNIT" : "float64",
    "TRANSACTION_ID" : "int64",
    "Product_Name" : "object",
    "Ingredient_Name" : "object",
    "Measure" : "object",
    "MME_Conversion_Factor" : "float64",
    "Combined_Labeler_Name" : "object",
#     "Revised_Company_Name" : "object", # was in original 2019 data
    "Reporter_family" : "object",
    "dos_str" : "float64",
    "MME" : "float64"
}
wp_data = pd.read_csv("../data/arcos-tx-statewide-itemized_downloadedjune9.csv", dtype=dtypes)

#### Checking for most common values, for nulls, etc:

In [ ]:
wp_data.info(null_counts=True)

In [ ]:
wp_data["BUYER_NAME"].value_counts().head()

In [ ]:
wp_data["ACTION_INDICATOR"].value_counts()

In [ ]:
# Let's try to parse through the time stamp on these transactions
wp_data["TRANSACTION_DATE"].head(10)

In [ ]:
wp_data["TRANSACTION_DATE"].sort_values().head()

In [ ]:
# Can see that we need to fill in preceeding zeros for months with 1 digit, 
# so each date has 8 digits
# First need to turn that column into strings
wp_data["TRANSACTION_DATE"] = wp_data["TRANSACTION_DATE"].astype('str')
wp_data["TRANSACTION_DATE"] = wp_data["TRANSACTION_DATE"].str.zfill(8)

In [ ]:
# Much better
wp_data["TRANSACTION_DATE"].head()

In [ ]:
# Now turning into a datetime object
wp_data["TRANSACTION_DATE"] = pd.to_datetime(wp_data["TRANSACTION_DATE"],
                                            format='%m%d%Y')

In [ ]:
# Success
wp_data["TRANSACTION_DATE"].head(10)

In [ ]:
# WP said they found 5,432,109,643 pills supplied to TX between 2006 and 2012
# Can see they arrived at that number using the Dosage Unit column
wp_data["DOSAGE_UNIT"].sum()

#### Creating smaller subsets of the Washington Post dataset:

In [ ]:
subset = wp_data.drop(columns=[
    "REPORTER_DEA_NO", "REPORTER_BUS_ACT", "REPORTER_ADDL_CO_INFO",
    "REPORTER_ADDRESS1", "REPORTER_ADDRESS2", "REPORTER_CITY",
    "REPORTER_COUNTY", "BUYER_DEA_NO", "BUYER_ADDL_CO_INFO", "DRUG_CODE",
    "NDC_NO", "UNIT", "ACTION_INDICATOR"])

In [ ]:
wp_data.head()

## Opioid Overdose Death Data

Source: [Texas Department of State Health Services](http://healthdata.dshs.texas.gov/Opioids/Deaths)

In [ ]:
dshs_2006 = pd.read_csv("../data/TXDSHS/2006_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")
dshs_2007 = pd.read_csv("../data/TXDSHS/2007_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")
dshs_2008 = pd.read_csv("../data/TXDSHS/2008_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")
dshs_2009 = pd.read_csv("../data/TXDSHS/2009_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")
dshs_2010 = pd.read_csv("../data/TXDSHS/2010_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")
dshs_2011 = pd.read_csv("../data/TXDSHS/2011_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")
dshs_2012 = pd.read_csv("../data/TXDSHS/2012_Commonly-Prescribed-Opioids_Deaths_by_County_data.csv")

In [ ]:
# Note that '---' isn't a null, it indicates the data was surpressed for 
# privacy purposes, and there were between 1 and 9 deaths 
dshs_2006.head()

In [ ]:
dshs_2006.shape

In [ ]:
# First, want to prepare these dataframes for a multi-index for modeling
# Creating a list of our yearly dataframes
dshs_dfs = [dshs_2006, dshs_2007, dshs_2008, dshs_2009, dshs_2010, 
            dshs_2011, dshs_2012]
# Just stacking them, using concat
dshs_all = pd.concat(dshs_dfs)

In [ ]:
dshs_all.shape

In [ ]:
dshs_all.head()

In [ ]:
# Now remember those --- are placeholders. The source gives the total number
# of deaths per year, so I can arrive at an average to get a good number
# to use in place of --- per year
# First, let's see how many of these placeholders we have - 626
year_placeholders = {}
for year in range(2006, 2013):
    year_df = dshs_all.loc[dshs_all["Year (copy)"] == year]
    placeholders = year_df.loc[year_df["Number of Deaths"] == "---"]
    year_placeholders[year] = len(placeholders)

In [ ]:
year_placeholders

In [ ]:
# Now, let's find the number of deaths we have in this database
dshs_all["Number of Deaths"].replace(to_replace="---", value=0, inplace=True)

In [ ]:
# Need to set the Number of Deaths column as an integer
dshs_all["Number of Deaths"] = dshs_all["Number of Deaths"].astype("int64")

In [ ]:
# Creating an empty dictionary, where we'll have each year as the key and 
# each yearly sum as the value
total_deaths = {}
for year in range(2006, 2013):
    year_df = dshs_all.loc[dshs_all["Year (copy)"] == year]
    year_sum = year_df["Number of Deaths"].sum()
    total_deaths[year] = year_sum

In [ ]:
total_deaths

In [ ]:
# These actual counts of the total number of opioid overdose deaths in Texas
# comes from the DSHS website
actual_deaths = {2006: 788, 2007: 767, 2008: 714, 2009: 717, 2010: 748,
                2011: 681, 2012: 649}

In [ ]:
# And now, let's find those annual averages for placeholder counties
average_placeholder = {}
for year in range(2006, 2013):
    death_difference = actual_deaths[year] - total_deaths[year]
    death_average = death_difference / year_placeholders[year]
    average_placeholder[year] = death_average

In [ ]:
average_placeholder

In [ ]:
# So now we know that, for all the placeholders in our data, 2.5 would be a 
# reasonable estimate to use in our analysis - so let's replace them
# Previously we replaced those --- with zeros, so need a new df to work with
dshs_for_csv = pd.concat(dshs_dfs)
dshs_for_csv.head()

In [ ]:
dshs_for_csv["Number of Deaths"].replace(to_replace="---", value="2.5", inplace=True)

In [ ]:
dshs_for_csv.head()

In [ ]:
# Writing to a csv
dshs_for_csv.to_csv(r"data/TXDSHS/TXDeaths_by_County_Data_NullsRemoved.csv",
                    index=False)

In [ ]:
# Sanity check
dshs_test = pd.read_csv(
    "data/TXDSHS/TXDeaths_by_County_Data_NullsRemoved.csv")
dshs_test.head()

#### Population Data

Source: [Texas State Library and Archives Commission](https://www.tsl.texas.gov/ref/abouttx/population.html), which links to the Census data to download.

Note that, per year, I am using the July county population estimates - even in years where there was a census conducted. This is for consistency, because if I need to use July estimates in other years I'd prefer to use it each year, not switching to the April 2010 census count and then back to July 2011 estimates (for example). 

In [ ]:
# Loading in the data for the 2000-2010 population estimates
# Defining column names and skipping some opening rows/footers because excel
pop_2000_2010 = pd.read_excel("../data/2000-2010_Population_Estimates_TX.xls",
                              names=["COUNTY", "APR_2000", "JUL_2000", 
                                     "JUL_2001", "JUL_2002", "JUL_2003", 
                                     "JUL_2004", "JUL_2005", "JUL_2006", 
                                     "JUL_2007", "JUL_2008", "JUL_2009", 
                                     "APR_2010", "JUL_2010"],
                              skiprows=[0, 1, 2, 3], skipfooter=8)

In [ ]:
pop_2000_2010.head()

In [ ]:
pop_2000_2010.tail(5)

In [ ]:
pop_2000_2010.info()

In [ ]:
# All of the counties have a period at the beginning
# We want them in the format "ANDERSON" not ".Anderson County"

# Removing the dot
pop_2000_2010["COUNTY"] = [x.strip('.') for x in pop_2000_2010["COUNTY"]]

# Removing " County"
pop_2000_2010["COUNTY"] = pop_2000_2010["COUNTY"].str.split(' County').str[0]

# Changing all to uppercase
pop_2000_2010["COUNTY"] = pop_2000_2010["COUNTY"].str.upper()

In [ ]:
# Dropping data from before my dataset, because I won't need it
# Also dropping 2010, since I'll use the updated and hopefully more accurate
# 2010 estimates from the more recent database
pop_2006_2009 = pop_2000_2010[[
    "COUNTY", "JUL_2006", "JUL_2007", "JUL_2008", "JUL_2009"]]

In [ ]:
# Much better
pop_2006_2009.head()

In [ ]:
# Now loading the data for 2010-2018 population estimates
pop_2010_2018 = pd.read_csv("../data/2010-2018_Population_Estimates_TX.csv",
                            header=1,
                            names=["EXT_ID", "ID", "COUNTY", "APR_2010_CEN",
                                   "APR_2010_BASE", "JUL_2010", "JUL_2011",
                                   "JUL_2012", "JUL_2013", "JUL_2014",
                                   "JUL_2015", "JUL_2016", "JUL_2017", 
                                   "JUL_2018"])

In [ ]:
pop_2010_2018.head()

In [ ]:
# Again, need to get the counties to be just the uppercase name

# Removing " County"
pop_2010_2018["COUNTY"] = pop_2010_2018["COUNTY"].str.split(' County').str[0]

# Changing all to uppercase
pop_2010_2018["COUNTY"] = pop_2010_2018["COUNTY"].str.upper()

pop_2010_2018.head()

In [ ]:
pop_2010_2012 = pop_2010_2018[["COUNTY", "JUL_2010", "JUL_2011", "JUL_2012"]]

In [ ]:
pop_2010_2012.head()

In [ ]:
# And now, a dataset of all the relevant population data!
pop_data = pop_2006_2009.merge(pop_2010_2012, on="COUNTY")
# Renaming columns for ease of use, since now they're all July estimates
pop_data.rename(columns={"JUL_2006": 2006, "JUL_2007": 2007,
                         "JUL_2008": 2008, "JUL_2009": 2009,
                         "JUL_2010": 2010, "JUL_2011": 2011,
                         "JUL_2012": 2012}, inplace=True)
pop_data.head()

#### Combining to arrive at a opioid death per capita figure, then deaths per 100k of population

In [ ]:
dshs_for_csv.head()

In [ ]:
# Some quick cleaning
dshs_clean = dshs_for_csv.copy()
# Only keeping the columns we want
dshs_clean.drop(columns=["Type of Death1", "Latitude (generated)", 
                         "Longitude (generated)"], inplace=True)
# Renaming the columns
dshs_clean.rename(columns={"County Name": "COUNTY",
                           "Number of Deaths": "NUM_DEATHS", 
                           "Year (copy)": "YEAR"}, inplace=True)
# Making all the county names uppercase
dshs_clean["COUNTY"] = dshs_clean["COUNTY"].str.upper()
# Making sure the values in the Number of Deaths column are floats
dshs_clean["NUM_DEATHS"] = dshs_clean["NUM_DEATHS"].astype("float")

dshs_clean.head()

In [ ]:
# Pivoting the table to make the columns each year
dshs_pivot = pd.pivot_table(dshs_clean, index="COUNTY",
                          columns="YEAR",
                          values="NUM_DEATHS")

In [ ]:
# Removing a weird index name, leftover from the pivot
dshs_pivot.rename_axis(None, axis=1, inplace=True)

In [ ]:
# Hooray, now it looks just like our population data table
dshs_pivot.head()

In [ ]:
pop_data.set_index("COUNTY", inplace=True)
pop_data.head()

In [ ]:
pop_data.loc[pop_data.index == "TERRELL"]

In [ ]:
dshs_pivot.loc[dshs_pivot.index == "TERRELL"]

In [ ]:
deaths_percapita = dshs_pivot / pop_data

In [ ]:
deaths_percapita.describe()

In [ ]:
pop_100k = pop_data / 100000

In [ ]:
deaths_per100k = dshs_pivot / pop_100k

In [ ]:
deaths_per100k.describe()

In [ ]:
deaths_per100k[2007].sort_values(ascending=False).head()

In [ ]:
wp_data.info()

In [ ]:
pills = wp_data[["BUYER_COUNTY", "TRANSACTION_DATE", "DOSAGE_UNIT"]].copy()

In [ ]:
pills.head()

In [ ]:
pills["YEAR"] = pills["TRANSACTION_DATE"].dt.year

In [ ]:
pills.drop(columns="TRANSACTION_DATE", inplace=True)
pills.rename(columns={"BUYER_COUNTY": "COUNTY"}, inplace=True)

In [ ]:
pills.head()

In [ ]:
# Alas, one specific county has a space in this dataframe, while all the 
# other datasets I have spell it without a space - let's fix that
pills["COUNTY"].replace(to_replace="DE WITT", value="DEWITT", inplace=True)

In [ ]:
# Pivoting the table to make the columns each year
pills_pivot = pd.pivot_table(pills, index="COUNTY", columns="YEAR",
                             values="DOSAGE_UNIT", aggfunc="sum")

In [ ]:
pills_pivot = pills_pivot.reindex_like(pop_data)

In [ ]:
# Filling nulls, when no pills were sent to that county
pills_pivot.fillna(0, inplace=True)
# Removing a weird index name, leftover from the pivot
pills_pivot.rename_axis(None, axis=1, inplace=True)

In [ ]:
pills_pivot.head()

In [ ]:
pills_percapita = pills_pivot / pop_data

In [ ]:
pills_percapita.describe()

In [ ]:
pills_percapita.head()

In [ ]:
pills_percapita.shape

## Visualizing

In [ ]:
# Need county identification numbers, which are Federal Information Processing
# Standard codes - which, luckily, were a part of one of the population csvs
county_id = {}
for obs in pop_2010_2018.index:
    county_id[pop_2010_2018["COUNTY"][obs]] = pop_2010_2018["ID"][obs]

In [ ]:
# Creating a new column for the County IDs
pills_percapita["COUNTY_ID"] = county_id.values()

In [ ]:
pills_percapita.head()

In [ ]:
pills_percapita.describe()

In [ ]:
pills_percapita.reset_index(inplace=True)

In [ ]:
pills_percapita.head()

## Creating a Better Dataframe to Export

In [ ]:
pills_pivot.reset_index(inplace=True)

In [ ]:
pills_pivot.head()

In [ ]:
pills_total_melted = pd.melt(pills_pivot, id_vars=["COUNTY"],
                           var_name="YEAR", value_name="TOTAL_PILLS")

In [ ]:
pills_total_melted.head()

In [ ]:
pop_data.head()

In [ ]:
pop_data.reset_index(inplace=True)
pop_melted = pd.melt(pop_data, id_vars=["COUNTY"],
                     var_name="YEAR", value_name="TOTAL_POPULATION")

In [ ]:
pop_melted.head()

In [ ]:
pills_percapita.head()

In [ ]:
pills_pc_melted = pd.melt(pills_percapita, id_vars=["COUNTY", "COUNTY_ID"],
                          var_name="YEAR", value_name="PILLS_PC")

In [ ]:
pills_pc_melted.head()

In [ ]:
dshs_pivot.head()

In [ ]:
dshs_pivot.reset_index(inplace=True)
deaths_melted = pd.melt(dshs_pivot, id_vars="COUNTY",
                        var_name="YEAR", value_name="TOTAL_DEATHS")

In [ ]:
deaths_melted.head()

In [ ]:
deaths_percapita.reset_index(inplace=True)

In [ ]:
deaths_percapita.head()

In [ ]:
deaths_pc_melted = pd.melt(deaths_percapita, id_vars=["COUNTY"],
                           var_name="YEAR", value_name="DEATHS_PC")

In [ ]:
deaths_pc_melted.head()

In [ ]:
deaths_per100k.head()

In [ ]:
deaths_per100k.reset_index(inplace=True)
deaths_p100k_melted = pd.melt(deaths_per100k, id_vars=["COUNTY"],
                              var_name="YEAR", value_name="DEATHS_PER_100K")

In [ ]:
deaths_p100k_melted.head()

In [ ]:
pop_melted.shape

In [ ]:
pills_total_melted.shape

In [ ]:
pills_pc_melted.shape

In [ ]:
deaths_melted.shape

In [ ]:
deaths_pc_melted.shape

In [ ]:
deaths_p100k_melted.shape

In [ ]:
county_data = pills_pc_melted[["COUNTY", "COUNTY_ID", "YEAR"]].copy()
county_data.head()

In [ ]:
county_data["TOTAL_POPULATION"] = pop_melted["TOTAL_POPULATION"]
county_data["TOTAL_PILLS"] = pills_total_melted["TOTAL_PILLS"]
county_data["PILLS_PER_CAPITA"] = pills_pc_melted["PILLS_PC"]
county_data["TOTAL_OVERDOSE_DEATHS"] = deaths_melted["TOTAL_DEATHS"]
county_data["DEATHS_PER_CAPITA"] = deaths_pc_melted["DEATHS_PC"]
county_data["DEATHS_PER_100K_PEOPLE"] = deaths_p100k_melted["DEATHS_PER_100K"]

In [ ]:
county_data.head()

## MORE DATA - Opioid Prescription Rates

From the [CDC's U.S. Opioid Prescribing Rate Maps](https://www.cdc.gov/drugoverdose/maps/rxrate-maps.html), which tracks retail opioid prescriptions dispensed per 100 persons per year.

My data is available as a [Google sheet](https://docs.google.com/spreadsheets/d/1fJWN3LYSLfiX_vkp4ONo0-dSyhKYmGvoYbmBOrp3PUk/edit?usp=sharing), after copy/pasting from the above source.

In [ ]:
prescribe_rate = pd.read_csv("../data/TexasCountyOpioidPrescribingRates(per100people)-From CDC.csv")

In [ ]:
prescribe_rate.rename(columns={"County": "COUNTY", 
                               "Prescribing Rate": "PRESCRIBE_RATE", 
                               "Year": "YEAR"}, inplace=True)
prescribe_rate["COUNTY"] = prescribe_rate["COUNTY"].str.upper()

In [ ]:
prescribe_rate.head()

In [ ]:
prescribe_rate.info()

In [ ]:
prescribe_rate.sort_values(by=["YEAR", "COUNTY"], inplace=True)

In [ ]:
prescribe_rate.reset_index(drop=True, inplace=True)

In [ ]:
prescribe_rate.head()

## And now, Unemployment Rates

From the [Bureau of Labor Statistics](https://data.bls.gov/lausmap/showMap.jsp)

My data is available in a [Google sheet](https://docs.google.com/spreadsheets/d/1BVljj8YRMTuZMQSyuwyd2LP2Y71-q-ryJSmkHgIgUbc/edit?usp=sharing), after copy/pasting from the above source.

In [ ]:
bls_data = pd.read_csv("../data/TexasCountyUnemploymentData-FromBLS.csv")

In [ ]:
bls_data.rename(columns={"County": "COUNTY", "July\n2006": "2006",
                         "July\n2007": "2007", "July\n2008": "2008",
                         "July\n2009": "2009", "July\n2010": "2010",
                         "July\n2011": "2011", "July\n2012": "2012"}, 
                inplace=True)
bls_data["COUNTY"] = bls_data["COUNTY"].str.upper()

In [ ]:
bls_data["COUNTY"] = bls_data["COUNTY"].str.split(' COUNTY').str[0]

In [ ]:
bls_data.head()

In [ ]:
bls_data_melted = pd.melt(bls_data, id_vars=["COUNTY"],
                          var_name="YEAR", value_name="UNEMPLOYMENT")

In [ ]:
bls_data_melted.head()

In [ ]:
county_data.head()

In [ ]:
county_data["PRESCRIPTION_RATE"] = prescribe_rate["PRESCRIBE_RATE"]
county_data["UNEMPLOYMENT"] = bls_data_melted["UNEMPLOYMENT"]

In [ ]:
county_data.head()

In [ ]:
# Writing to CSV
# county_data.to_csv(r"../data/TX_County_By_Year.csv", index=False)

In [ ]:
# Sanity check
county_test = pd.read_csv("../data/TX_County_By_Year.csv")

In [ ]:
county_test.head()